In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Local imports
from data_fetcher import DataFetcher
# from preprocessing import Processor

In [3]:
# Define some string constants for easy typing
SAMPLE_DATA_BY_SITE = 'sampleData/bySite'
SAMPLE_DATA_BY_COUNTY = 'sampleData/byCounty'
SAMPLE_DATA_BY_STATE = 'sampleData/byState'
SAMPLE_DATA_BY_BOX = 'sampleData/byBox'
SAMPLE_DATA_BY_CBSA = 'sampleData/byCBSA'

LIST_STATES = 'list/states'
LIST_COUNTIES_BY_STATE = 'list/countiesByState'
LIST_SITES_BY_COUNTY = 'list/sitesByCounty'
LIST_CBSAs = 'list/cbsas'
LIST_PARAM_CLASSES = 'list/classes'
LIST_PARAM_IN_CLASS = 'list/parametersByClass'

In [4]:
datafetcher = DataFetcher()

I'm going to be investigating PM2.5 runtime (so if I can make the annual check way faster)

A normal pull looks like this:

In [28]:
north_main = datafetcher.create_dataset(20170101, 20171231, '1103', '037', '06')
north_main

No hourly data for PM2.5 - Local Conditions


,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,Wind Direction - Resultant,Wind Speed - Resultant,Outdoor Temperature,Relative Humidity,Solar radiation,Ultraviolet radiation,Barometric pressure
datetime,,,,,,,,,,
2017-01-01 00:00:00,0.4695,16.7,0.010,353.0,1.7,45.4,80.0,0.0,0.0,1000.0
2017-01-01 01:00:00,0.5720,22.7,0.004,27.0,4.7,45.3,80.0,0.0,0.0,1001.0
2017-01-01 02:00:00,0.4530,17.0,0.011,22.0,3.0,45.6,76.0,0.0,0.0,1000.0
2017-01-01 03:00:00,0.4625,19.5,0.006,19.0,1.8,45.1,79.0,0.0,0.0,1000.0
2017-01-01 04:00:00,0.4885,20.9,0.003,32.0,2.2,44.2,83.0,0.0,0.0,1000.0
...,...,...,...,...,...,...,...,...,...,...
2017-12-31 19:00:00,0.4300,15.7,0.023,308.0,1.8,56.4,91.0,0.0,0.0,1009.0
2017-12-31 20:00:00,0.6460,21.5,0.013,8.0,2.1,55.7,92.0,0.0,0.0,1009.0
2017-12-31 21:00:00,0.8305,28.2,0.003,17.0,1.8,54.6,91.0,0.0,0.0,1009.0


Ok so now I want to try to fix the situation where PM2.5 is pulled instead!

This is one situation, where the hourly is just pulled second:

In [10]:
#4004 is long beach
long_beach_small = datafetcher.get_data(SAMPLE_DATA_BY_SITE, 88101, 20190410, 20190415, df = True, nparams={'state':'06', 'county':'037', 'site':'4004'})
long_beach_small = long_beach_small[["date_local", "time_local", "sample_measurement", "sample_duration"]]
long_beach_small

,date_local,time_local,sample_measurement,sample_duration
0,2019-04-10,00:00,3.9,24 HOUR
1,2019-04-11,00:00,11.8,24 HOUR
2,2019-04-12,00:00,10.6,24 HOUR
3,2019-04-13,00:00,10.0,24 HOUR
4,2019-04-14,00:00,9.7,24 HOUR
...,...,...,...,...
145,2019-04-15,19:00,8.0,1 HOUR
146,2019-04-15,20:00,10.7,1 HOUR
147,2019-04-15,21:00,11.7,1 HOUR
148,2019-04-15,22:00,6.5,1 HOUR


Now we can see what create_database() does to this data (via the processor)

In [11]:
long_beach_reg = datafetcher.create_dataset(20190410, 20190415, site=4004, county='037', state='06', processed=True, verbose=False)
long_beach_reg

No data for Carbon monoxide
No data for Nitrogen dioxide (NO2)
No data for Ozone
No data for Solar radiation
No data for Ultraviolet radiation


,PM2.5 - Local Conditions,Wind Direction - Resultant,Wind Speed - Resultant,Relative Humidity,Barometric pressure
datetime,,,,,
2019-04-10 00:00:00,3.9,324.0,13.1,29.0,1013.0
2019-04-10 01:00:00,NaN,323.0,12.3,32.0,1013.0
2019-04-10 02:00:00,NaN,319.0,10.0,33.0,1013.0
2019-04-10 03:00:00,NaN,318.0,9.2,34.0,1013.0
2019-04-10 04:00:00,NaN,328.0,11.7,33.0,1014.0
...,...,...,...,...,...
2019-04-15 19:00:00,NaN,304.0,0.7,72.0,1008.0
2019-04-15 20:00:00,NaN,124.0,1.9,72.0,1009.0
2019-04-15 21:00:00,NaN,179.0,4.6,71.0,1009.0


OK so this is a weird twist because before there was data there lmao for 4/15. But I don't think it's going to change anything about how I fix it!

In [12]:
# now trying to do only hourly columns
# df.loc[df['col1'] == value]

long_beach_small.loc[long_beach_small["sample_duration"] == "1 HOUR"]

,date_local,time_local,sample_measurement,sample_duration
6,2019-04-10,00:00,9.2,1 HOUR
7,2019-04-10,01:00,6.0,1 HOUR
8,2019-04-10,02:00,3.3,1 HOUR
9,2019-04-10,03:00,2.6,1 HOUR
10,2019-04-10,04:00,2.1,1 HOUR
...,...,...,...,...
145,2019-04-15,19:00,8.0,1 HOUR
146,2019-04-15,20:00,10.7,1 HOUR
147,2019-04-15,21:00,11.7,1 HOUR
148,2019-04-15,22:00,6.5,1 HOUR


In [14]:
# doing an example to make an empty df 
# this is to check if append and the regular function will still work
empty = long_beach_small.loc[long_beach_small["sample_duration"] == "0"]
empty
dfs = []
dfs.append(empty)
dfs

[Empty DataFrame
 Columns: [date_local, time_local, sample_measurement, sample_duration]
 Index: []]

In [17]:
long_beach_reg = datafetcher.create_dataset(20190410, 20190415, site=4004, county='037', state='06', processed=True, verbose=False)
long_beach_reg

No data for Carbon monoxide
No data for Nitrogen dioxide (NO2)
No data for Ozone
No data for Solar radiation
No data for Ultraviolet radiation


,PM2.5 - Local Conditions,Wind Direction - Resultant,Wind Speed - Resultant,Relative Humidity,Barometric pressure
datetime,,,,,
2019-04-10 00:00:00,6.55,324.0,13.1,29.0,1013.0
2019-04-10 01:00:00,6.00,323.0,12.3,32.0,1013.0
2019-04-10 02:00:00,3.30,319.0,10.0,33.0,1013.0
2019-04-10 03:00:00,2.60,318.0,9.2,34.0,1013.0
2019-04-10 04:00:00,2.10,328.0,11.7,33.0,1014.0
...,...,...,...,...,...
2019-04-15 19:00:00,8.00,304.0,0.7,72.0,1008.0
2019-04-15 20:00:00,10.70,124.0,1.9,72.0,1009.0
2019-04-15 21:00:00,11.70,179.0,4.6,71.0,1009.0


Yay!!! Ok it now works :)
Let's try for LA North Mainstreet! (since we know they don't have hourly)

In [24]:
northmain_small = datafetcher.get_data(SAMPLE_DATA_BY_SITE, 88101, 20190410, 20190415, df = True, nparams={'state':'06', 'county':'037', 'site':'1103'})
northmain_small = northmain_small[["date_local", "time_local", "sample_measurement", "sample_duration"]]
northmain_small

,date_local,time_local,sample_measurement,sample_duration
0,2019-04-10,00:00,2.3,24 HOUR
1,2019-04-11,00:00,10.1,24 HOUR
2,2019-04-12,00:00,9.3,24 HOUR
3,2019-04-13,00:00,10.4,24 HOUR
4,2019-04-14,00:00,11.7,24 HOUR
5,2019-04-15,00:00,11.2,24 HOUR
6,2019-04-15,00:00,10.5,24 HOUR


Plot twist: the previous version with loc actually didn't return empty when there were no results, so I adjusted it to this, which works!

In [27]:
one_hour = northmain_small[northmain_small['sample_duration'] == "1 HOUR"]
# northmain_small = northmain_small.drop(['sample_duration'], axis=1)
one_hour

,date_local,time_local,sample_measurement,sample_duration


In [26]:
northmain_small

,date_local,time_local,sample_measurement,sample_duration
0,2019-04-10,00:00,2.3,24 HOUR
1,2019-04-11,00:00,10.1,24 HOUR
2,2019-04-12,00:00,9.3,24 HOUR
3,2019-04-13,00:00,10.4,24 HOUR
4,2019-04-14,00:00,11.7,24 HOUR
5,2019-04-15,00:00,11.2,24 HOUR
6,2019-04-15,00:00,10.5,24 HOUR


In [31]:
northmain_reg = datafetcher.create_dataset(20190410, 20190415, site=1103, county='037', state='06', processed=True, verbose=False)
northmain_reg

No hourly data for PM2.5 - Local Conditions


,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,Wind Direction - Resultant,Wind Speed - Resultant,Outdoor Temperature,Relative Humidity,Solar radiation,Ultraviolet radiation,Barometric pressure
datetime,,,,,,,,,,
2019-04-10 00:00:00,0.2000,1.8,0.043,324.0,7.4,63.2,27.0,0.0,0.0,1005.0
2019-04-10 01:00:00,0.2000,1.3,0.043,312.0,7.2,62.5,29.0,0.0,0.0,1005.0
2019-04-10 02:00:00,0.2000,1.4,0.041,314.0,8.1,61.7,30.0,0.0,0.0,1005.0
2019-04-10 03:00:00,0.2000,2.6,0.040,320.0,10.3,62.0,28.0,0.0,0.0,1005.0
2019-04-10 04:00:00,0.2000,3.5,0.039,323.0,12.7,62.1,28.0,0.0,0.0,1005.0
...,...,...,...,...,...,...,...,...,...,...
2019-04-15 19:00:00,0.1880,7.1,0.033,265.0,5.1,56.4,75.0,0.0,0.0,1000.0
2019-04-15 20:00:00,0.2535,11.2,0.030,188.0,3.7,56.6,73.0,0.0,0.0,1001.0
2019-04-15 21:00:00,0.2610,17.3,0.023,191.0,3.2,56.8,73.0,0.0,0.0,1001.0


Alright now that that's working, we need a way to ensure that the data we're pulling is coming from the right place!

I'm going to call for a test in concat, because that's where the raw data is

First, as a reminder of the raw data we're working with:

In [6]:
get_data = datafetcher.get_data(SAMPLE_DATA_BY_SITE, 44201, 20170618, 20170618, df = True, nparams={'state':'06', 'county':'037', 'site':'1103'})
get_data

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,date_local,...,detection_limit,uncertainty,qualifier,method_type,method,method_code,state,county,date_of_last_change,cbsa_code
0,06,037,1103,44201,1,34.06659,-118.22688,WGS84,Ozone,2017-06-18,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
1,06,037,1103,44201,1,34.06659,-118.22688,WGS84,Ozone,2017-06-18,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
2,06,037,1103,44201,1,34.06659,-118.22688,WGS84,Ozone,2017-06-18,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
3,06,037,1103,44201,1,34.06659,-118.22688,WGS84,Ozone,2017-06-18,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
4,06,037,1103,44201,1,34.06659,-118.22688,WGS84,Ozone,2017-06-18,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
5,06,037,1103,44201,1,34.06659,-118.22688,WGS84,Ozone,2017-06-18,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
6,06,037,1103,44201,1,34.06659,-118.22688,WGS84,Ozone,2017-06-18,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
7,06,037,1103,44201,1,34.06659,-118.22688,WGS84,Ozone,2017-06-18,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
8,06,037,1103,44201,1,34.06659,-118.22688,WGS84,Ozone,2017-06-18,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
9,06,037,1103,44201,1,34.06659,-118.22688,WGS84,Ozone,2017-06-18,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080


In [12]:
get_data = get_data.drop(['state_code', 'county_code', 'site_number', 'parameter_code', 'poc', 'datum', 'parameter', 'sample_duration_code'], axis=1)
get_data

,latitude,longitude,date_local,time_local,date_gmt,time_gmt,sample_measurement,units_of_measure,units_of_measure_code,sample_duration,...,detection_limit,uncertainty,qualifier,method_type,method,method_code,state,county,date_of_last_change,cbsa_code
0,34.06659,-118.22688,2017-06-18,00:00,2017-06-18,08:00,0.005,Parts per million,007,1 HOUR,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
1,34.06659,-118.22688,2017-06-18,01:00,2017-06-18,09:00,0.001,Parts per million,007,1 HOUR,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
2,34.06659,-118.22688,2017-06-18,02:00,2017-06-18,10:00,0.001,Parts per million,007,1 HOUR,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
3,34.06659,-118.22688,2017-06-18,03:00,2017-06-18,11:00,0.001,Parts per million,007,1 HOUR,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
4,34.06659,-118.22688,2017-06-18,04:00,2017-06-18,12:00,0.001,Parts per million,007,1 HOUR,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
5,34.06659,-118.22688,2017-06-18,05:00,2017-06-18,13:00,0.006,Parts per million,007,1 HOUR,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
6,34.06659,-118.22688,2017-06-18,06:00,2017-06-18,14:00,0.010,Parts per million,007,1 HOUR,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
7,34.06659,-118.22688,2017-06-18,07:00,2017-06-18,15:00,0.015,Parts per million,007,1 HOUR,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
8,34.06659,-118.22688,2017-06-18,08:00,2017-06-18,16:00,0.024,Parts per million,007,1 HOUR,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080
9,34.06659,-118.22688,2017-06-18,09:00,2017-06-18,17:00,0.041,Parts per million,007,1 HOUR,...,0.005,None,None,FEM,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,087,California,Los Angeles,2017-09-28,31080


In [15]:
#normally we insert a full date range
# in the graph it's the form 2017-06-19
# meanwhile we do 20170619
date = '20170618'
date = date[0:4] + '-' + date[4:6] + '-' + date[6:]
date

is_it = (get_data.at[0, 'date_gmt'] != date)
is_it
# so they aren't equal haha

False

It might be good to check the following parameters:

date_local
parameter
site_number 

In [16]:
# for time, comparing data/times
# or check date_local!
# it's now implemented so!

northmain_reg = datafetcher.create_dataset(20190410, 20190415, site=1103, county='037', state='06', processed=True, verbose=False)
northmain_reg

No hourly data for PM2.5 - Local Conditions


,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,Wind Direction - Resultant,Wind Speed - Resultant,Outdoor Temperature,Relative Humidity,Solar radiation,Ultraviolet radiation,Barometric pressure
datetime,,,,,,,,,,
2019-04-10 00:00:00,0.2000,1.8,0.043,324.0,7.4,63.2,27.0,0.0,0.0,1005.0
2019-04-10 01:00:00,0.2000,1.3,0.043,312.0,7.2,62.5,29.0,0.0,0.0,1005.0
2019-04-10 02:00:00,0.2000,1.4,0.041,314.0,8.1,61.7,30.0,0.0,0.0,1005.0
2019-04-10 03:00:00,0.2000,2.6,0.040,320.0,10.3,62.0,28.0,0.0,0.0,1005.0
2019-04-10 04:00:00,0.2000,3.5,0.039,323.0,12.7,62.1,28.0,0.0,0.0,1005.0
...,...,...,...,...,...,...,...,...,...,...
2019-04-15 19:00:00,0.1880,7.1,0.033,265.0,5.1,56.4,75.0,0.0,0.0,1000.0
2019-04-15 20:00:00,0.2535,11.2,0.030,188.0,3.7,56.6,73.0,0.0,0.0,1001.0
2019-04-15 21:00:00,0.2610,17.3,0.023,191.0,3.2,56.8,73.0,0.0,0.0,1001.0


# OTHER SECTION OF TESTING 

In [14]:
# testing how sample days works
sample_days = [datafetcher.sample_day_in_year(year, year + 10000) for year in range(20170101, 20200101, 50000)]
print(sample_days)

[('20171213', '20171214')]


So every time it prints out just one tuple of 2 days in a row to sample in a year
I think it almost always does the every first year and then every 5 years from that 
Concern: doesn't check first day so there's a chance there's no data there.
Maybe add that to a big checker function, and the function returns all the tests that the data passes
(geographical region, etc.)
Then it instructs you to different functions to print out graphs of the data 

In [13]:
# testing how sample days works
sample_days = [datafetcher.sample_day_in_year(year, year + 10000) for year in range(20000101, 20200101, 50000)]
print(sample_days)

[('20000225', '20000226'), ('20051001', '20051002'), ('20100920', '20100921'), ('20150721', '20150722')]


In [22]:
# so this looks like it'll take at least 25 mins to run the way it's currently set up
datafetcher.find_best_location(state='06', county='037', bdate=20170101, edate=20200101)

Searching county 037 in state 06... Found 51 sites.
[('20170722', '20170723')]
Finished site 0002, Azusa
Finished site 0016, Glendora
Finished site 0018, El Monte
Finished site 0030, SB25 trailer at Hollenbeck School
Finished site 0031, Wilmington-N. Mahar Ave
Finished site 0113, West Los Angeles
Finished site 0201, Carson
Finished site 0202, Commerce-Ayers Ave
Finished site 0203, City of Industry-Volkswagon
Finished site 0204, City of Industry-Whitco
Finished site 0205, Commerce-AT&SF RR
Finished site 0206, SITE IS LOCATED ONE HALF MILE EAST OF THE I-57/I-60 INTERCHANGE
Finished site 1002, Burbank
Finished site 1101, UNKNOWN COORDINATE LOCATION
Finished site 1102, ON BUILDING
Finished site 1103, Los Angeles-North Main Street
Finished site 1105, West Los Angeles-Robertson Blvd
Finished site 1201, Reseda
Finished site 1301, Lynwood
Finished site 1302, Compton
Finished site 1402, Van Nuys Airport
Finished site 1403, Uddelholm (Trojan Battery)
Finished site 1404, Closet World (Quemetco)
F

{'Data': {'Azusa': [[True],
   [True],
   [True],
   [False],
   [True],
   [True],
   [True],
   [True],
   [False],
   [False],
   [True],
   [True],
   [True]],
  'Glendora': [[True],
   [True],
   [True],
   [False],
   [True],
   [True],
   [False],
   [True],
   [False],
   [False],
   [False],
   [True],
   [True]],
  'El Monte': [[False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False]],
  'SB25 trailer at Hollenbeck School': [[False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False]],
  'Wilmington-N. Mahar Ave': [[False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False],
   [False]],
  'West Los Angeles': [[True],
   [True],
   [True],
   [False],
   [True],
   [True],
   [False],
   [False],
   [False],
   [F

Ok so I have no idea what to do with that output haha
I honestly don't know why it looks to complicated

In [ ]:
# res = {}
# res['Data'] = {}
# res['Metadata'] = {'dates':sample_days, 'codes':codes}

# How to store the data so I'm not pulling
# r = datafetcher.find_best_location()
# with open('data.json', 'w') as fp:
#     json.dump(r, fp)

# ok so in this library we aleady have the data in a json for 2000 to 2021
with open('data.json', 'r') as f:
  r = json.load(f)

data = r['Data']
metadata = r['Metadata']

num_codes = len(data['Azusa']) # Any key in result dict
num_years = len(data['Azusa'][0])

yearly_results = {site:[sum([data[site][code][year] for code in range(num_codes)]) for year in range(num_years)] for site in data}
yearly_best = {site: ([date for date, val in enumerate(yearly_results[site]) if val == max(yearly_results[site])], max(yearly_results[site])) for site in yearly_results}
yearly_best_sorted = sorted(yearly_best.items(), key=lambda x : x[1][1], reverse=True)
yearly_best_sorted[:5] # NOTE: (year whree most measurements weree takeen, most mausrements taken)